In [1]:
from io import StringIO
import pandas as pd
from sqlalchemy import create_engine, text
engine = create_engine('sqlite://', echo=False)
import sqlite3
import contextlib

def df_to_database(df, name):
    """
       Deze functie maakt een database van een dataframe.
       Parameters:
                  df  = de dataframe waarvan je een database tabel wil maken.
                         
                  name = De naam die je de database wil geven.
                         Zorg ervoor dat deze parameter een string is.
    """
    df.to_sql(name, con=engine)
    
    display(df)

def string_to_df(string):
    """
       Deze functie maakt van een string een dataframe,
       zolang het standaard CSV formaat wordt aangehouden.
       Parameters:
                  string = de string waarvan je een dataframe wil maken.
    """
    return pd.read_csv(StringIO(string))

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """

    query_output = engine.execute(query).fetchall()
    column_names = engine.execute(query)._metadata.keys
    return pd.DataFrame(query_output, columns = column_names)


sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()


query_r = "SELECT * FROM sqlite_sequence" # read query


sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()
df = pd.read_sql_query(query_r,sql_connect)


import contextlib
@contextlib.contextmanager
def db_read_only(file):
    sql_connect = sqlite3.connect('file:'+file+'?mode=ro',uri=True)
    cursor = sql_connect.cursor()
    yield sql_connect
    sql_connect.commit()
    sql_connect.close()
    
    
file = 'database.sqlite'
with db_read_only(file) as ro:
    data = pd.read_sql_query(query_r, ro)
    
    
df_list = []
with db_read_only(file) as ro:
    for table in data['name']:
        query_r = 'SELECT * FROM ' + str(table)
        data = pd.read_sql_query(query_r, ro)
        df_list.append(data)


In [11]:
team = df_list[0]
country = df_list[1]
league = df_list[2]
match = df_list[3]
player = df_list[4]
player_attributes = df_list[5]
team_attributes = df_list[6]


# OPDRACHT 1

Er zijn twee verschillende soorten supervised learning. Je hebt een regressie en een classificatie. Een regressie wordt gebruikt als er sprake is van continue data. Er is sprake van een classificatie als er gebruik wordt gemaakt van categoriële data. Onze data wordt ingedeeld in 3 categoriën; win, defeat en draw. Bij onze data is er dus sprake van een classificatie.

# OPDRACHT 2

In [35]:
#df3000 is een dataframe waar we uitrekenen of het uit of thuis team wint. 
df3000 = match['home_team_goal'] - match['away_team_goal']
l_list = []

#In de list comprehension geven we aan elke value in df3000 het label 'win', 'defeat' of draw.
list_comp = ['Win' if value > 0  else 'Defeat' if value < 0  else 'Draw' for value in df3000]
match['win_or_lose'] = list_comp
match2 = match[['id', 'country_id', 'league_id', 'season', 'match_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'win_or_lose']]
match2

,id,country_id,league_id,season,match_api_id,away_team_api_id,home_team_goal,away_team_goal,win_or_lose
0,1,1,1,2008/2009,492473,9993,1,1,Draw
1,2,1,1,2008/2009,492474,9994,0,0,Draw
2,3,1,1,2008/2009,492475,8635,0,3,Defeat
3,4,1,1,2008/2009,492476,9998,5,0,Win
4,5,1,1,2008/2009,492477,9985,1,3,Defeat
...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,1992091,10191,1,0,Win
25975,25976,24558,24558,2015/2016,1992092,10199,1,2,Defeat
25976,25977,24558,24558,2015/2016,1992093,10179,2,0,Win
25977,25978,24558,24558,2015/2016,1992094,10243,0,0,Draw


# OPDRACHT 3

- Overall_rating: Hier zijn alle ratings in samengevoegd. 
- Potential: Hier zie je per speler wat het hoogste level van hun carriere kan worden.  
- Ball_control: Hier zie je per speler hoe goed zij controle hebben over de bal. 
- Dribbling: Hier zie je per speler hoe goed zij kunnen dribbelen (op een hoog tempo kleine stappen zetten met de bal aan hun   voet)

Wij verwachten dat bij hogere waardes de kans op winst groter is.

# OPDRACHT 4

In [117]:
af3 = adding_overall_rating('overall_rating', '_rating')

s_home = af3[af3['season']== '2015/2016'].filter(like = 'home').filter(like = "rating")
rows_list = []
for i, row in s_home.iterrows():
    row_dict = {'Index': i, 'Home_players': row.mean()}
    rows_list.append(row_dict)

home= pd.DataFrame(rows_list)
display(home)

s_away = af3[af3['season']== '2015/2016'].filter(like = 'away').filter(like = "rating")
rows_list1 = []
for i, row in s_away.iterrows():
    row_dict = {'Index': i, 'Away_players': row.mean()}
    rows_list1.append(row_dict)

away = pd.DataFrame(rows_list1)
display(away)

s_home = af3.filter(like = 'home')
s_home

s_15_16 = home.merge(away, on = "Index")
s_15_16

new_df = pd.DataFrame(s_home["home_team_api_id"])
new_df

merged = s_15_16.merge(new_df, left_on = "Index", right_index = True)
grouped_teams = merged.groupby("home_team_api_id")[["Home_players", "Away_players"]].mean().reset_index()
grouped_teams

,Index,Home_players
0,1488,64.727273
1,1489,65.545455
2,1490,68.727273
3,1491,67.454545
4,1492,65.727273
...,...,...
3321,25974,59.909091
3322,25975,59.272727
3323,25976,64.818182
3324,25977,59.500000


,Index,Away_players
0,1488,68.818182
1,1489,68.000000
2,1490,64.818182
3,1491,66.181818
4,1492,65.272727
...,...,...
3321,25974,60.545455
3322,25975,62.818182
3323,25976,64.909091
3324,25977,62.363636


,home_team_api_id,Home_players,Away_players
0,1601,61.212121,61.957576
1,1773,65.186061,66.569697
2,1957,60.807273,62.147340
3,2182,65.060606,61.834545
4,2186,61.430303,61.995152
...,...,...,...
183,158085,67.593048,68.995722
184,177361,60.303030,62.406397
185,188163,65.639691,68.838503
186,208931,67.425837,72.604785


In [94]:
grouped_teams["Difference"] = grouped_teams["Home_players"] - grouped_teams["Away_players"]
grouped_teams

,home_team_api_id,Home_players,Away_players,Difference
0,1601,61.212121,61.957576,-0.745455
1,1773,65.186061,66.569697,-1.383636
2,1957,60.807273,62.147340,-1.340067
3,2182,65.060606,61.834545,3.226061
4,2186,61.430303,61.995152,-0.564848
...,...,...,...,...
183,158085,67.593048,68.995722,-1.402674
184,177361,60.303030,62.406397,-2.103367
185,188163,65.639691,68.838503,-3.198812
186,208931,67.425837,72.604785,-5.178947


In [95]:
opdr4 = match.merge(grouped_teams, on = "home_team_api_id")
opdr4

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,win_or_lose,home_player_1_rating,home_player_2_rating,home_player_3_rating,home_player_4_rating,home_player_5_rating,home_player_6_rating,home_player_7_rating,home_player_8_rating,home_player_9_rating,home_player_10_rating,home_player_11_rating,away_player_1_rating,away_player_2_rating,away_player_3_rating,away_player_4_rating,away_player_5_rating,away_player_6_rating,away_player_7_rating,away_player_8_rating,away_player_9_rating,away_player_10_rating,away_player_11_rating,Home_players,Away_players,Difference
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.0,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.30,3.75,NaN,NaN,NaN,1.70,3.3,4.33,1.90,3.3,4.0,1.65,3.4,4.5,1.78,3.25,4.00,1.73,3.40,4.20,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.284848,66.575758,1.709091
1,29,1,1,2008/2009,12,2008-11-15 00:00:00,492583,9987,9999,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.25,5.25,10.0,1.23,5.00,10.00,1.30,4.2,8.0,1.25,4.50,10.00,NaN,NaN,NaN,1.25,4.8,9.00,1.27,5.5,10.0,1.25,5.0,9.0,1.27,5.00,8.50,1.25,5.00,9.00,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.284848,66.575758,1.709091
2,47,1,1,2008/2009,14,2008-11-29 00:00:00,492651,9987,9984,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,4.5,1.65,3.45,4.90,1.65,3.4,4.2,1.72,3.40,4.00,NaN,NaN,NaN,1.70,3.4,4.20,1.67,3.5,5.0,1.70,3.4,4.5,1.70,3.40,4.40,1.70,3.40,4.33,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.284848,66.575758,1.709091
3,65,1,1,2008/2009,16,2008-12-13 00:00:00,492713,9987,9986,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [111]:
opdr5 = opdr4[["Home_players", "Difference", "win_or_lose"]]
opdr5

,Home_players,Difference,win_or_lose
0,68.284848,1.709091,Draw
1,68.284848,1.709091,Draw
2,68.284848,1.709091,Win
3,68.284848,1.709091,Win
4,68.284848,1.709091,Win
...,...,...,...
20474,59.453816,-3.354265,Defeat
20475,59.453816,-3.354265,Win
20476,59.453816,-3.354265,Defeat
20477,59.453816,-3.354265,Defeat


# OPDRACHT 5

Logistieke Regressie, dit gebruik je bij categorische waardes. Het is een eenvoudig en interpreteerbaar  algoritme. Logistieke Regressie is vooral handig te gebruiken als je historische waardes hebt, deze hebben wij. Hiermee train je het model. Deze kan je uiteindelijk gebruiken om nieuwe wedstrijden te voorspellen

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X = opdr5[["Home_players", "Difference"]].values
y = opdr5["win_or_lose"].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=60)
new_df = LogisticRegression(random_state = 42)
new_df.fit(X_train, y_train)
display(new_df.score(X_test, y_test))

0.495849609375

__Na het evalueren van de score uit de logistieke regressie valt het ons op dat de waarde redelijk hoog ligt als het gaat om het voorspellen van voetbal uitslagen. De waardes kunnen nooit zicht rond de 1 liggen, omdat het simpelweg onmogelijk is om voor een wedstrijd te voorspellen wie er gaat winnen.__

K-Nearest Neighbors: Dit is een classificatie-algoritme. Wij hebben in opdracht 1 duidelijk gemaakt dat wij hier te maken hebben met classificatie. Het doet ook geen aannames over de verdeling van de gegevens.

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 60)


from sklearn.neighbors import KNeighborsClassifier

train_accuracies = {}
test_accuracies = {}
neighbors = range(1, 11)

for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors = neighbor)
    knn.fit(X_train, y_train)
    train_accuracies[neighbor] = knn.score(X_train, y_train)
    test_accuracies[neighbor] = knn.score(X_test, y_test)
print(neighbors, train_accuracies, test_accuracies)

D:\School\Anaconda\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\School\Anaconda\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `k

range(1, 11) {1: 0.4171495540074224, 2: 0.35405950908262257, 3: 0.427436682075656, 4: 0.42717624845367536, 5: 0.44664366169672504, 6: 0.45269874340777394, 7: 0.4469040953187056, 8: 0.443583566638453, 9: 0.4582329578748616, 10: 0.4694967120255225} {1: 0.4125, 2: 0.3400390625, 3: 0.410546875, 4: 0.408203125, 5: 0.427734375, 6: 0.4265625, 7: 0.43125, 8: 0.430078125, 9: 0.435546875, 10: 0.4416015625}


D:\School\Anaconda\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
